<a href="https://colab.research.google.com/github/likeJoseph/helloworld/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install mysql-connector-python



---



In [25]:
import mysql.connector
import sys
import pandas as pd
import matplotlib.pyplot as plt
# import mysql

**database 연결 정보 입력**

In [28]:
# Connect to the database
HOST = "147.46.15.238" # database 를 생성/연결할 서버 IP
PORT = "7000"
USER = "DBilgyo_chong"
PASSWD = "DBilgyo_chong"
DB = "DBilgyo_chong"

# mysql 을 서버에서 활용할 수 있도록 연결해준다.
connection = mysql.connector.connect(
    host=HOST,
    port=PORT,
    user=USER,
    passwd=PASSWD,
    db=DB,
    autocommit=True  # to create table permanently
)

# Cursor 를 통해 서버의 database 와 커뮤니케이트 한다.
cur = connection.cursor(dictionary=True)

# SQL query 를 받아 해당 query를 보내고 그 결과 값을 dataframe으로 저장해 return 해주는 함수
def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

In [29]:
# ratings, item_similarity, user_similarity
# test connection
query = "SHOW tables;"     # 서버의 database 에 보내고 싶은 sql query
cur.execute(query)         # cur.execute() 을 통해 query 를 서버로 송출
a = cur.fetchall()         # cur.fetchall() 을 통해 query 실행 후 결과값 반환
print(pd.DataFrame(a))

   Tables_in_DBilgyo_chong
0                 customer
1              geolocation
2          item_similarity
3               order_info
4               order_item
5            order_payment
6             order_review
7                  product
8                  ratings
9                   seller
10         user_similarity


## <b> 1. Popularity-based Recommendation (Popularity by rating count)

In [30]:
query = '''
SELECT * FROM ratings
'''
df = get_output(query)

In [31]:
# 1번 답
rec_num = 10

query = '''SELECT item, COUNT(user) AS count
      FROM ratings
WHERE rating IS NOT NULL AND item >= 150 AND item <350
GROUP BY item
ORDER BY count DESC, item
LIMIT {}'''.format(rec_num)

get_output(query)


,item,count
0,203,194
1,232,187
2,200,167
3,233,155
4,208,153
5,227,152
6,244,152
7,172,151
8,280,149
9,234,148




---

## <B> 2. Popularity by average rating


In [32]:
# 답2
rec_num = 10

query = '''
SELECT item, ROUND(AVG(P),4) AS prediction
FROM (SELECT user, item, (rating-min)/(max-min) AS P FROM ratings
      LEFT JOIN
     (SELECT user, MAX(rating) AS max, MIN(rating) AS min FROM ratings GROUP BY user) R USING(user)) R2
WHERE item >= 150 AND item <350
GROUP BY item
ORDER BY prediction DESC, item
LIMIT {}
'''.format(rec_num)

get_output(query)

,item,prediction
0,316,0.8666
1,203,0.8634
2,307,0.8347
3,290,0.8321
4,310,0.8311
5,306,0.8268
6,295,0.8265
7,157,0.8244
8,292,0.8227
9,344,0.8163




---

## <B> 3. IBCF

In [33]:
# terminal 입력 부분
user_id = 10
rec_cnt = 10
rec_num = 3

(2) 아이템마다 가장 유사도가 높은 이웃 K개를 구한다.

In [34]:
query = '''
SELECT *
FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking
FROM item_similarity) R
WHERE ranking <= {K}
'''.format(K = 5)
get_output(query)

,item_1,item_2,sim,ranking
0,0,3,0.8,1
1,0,7,0.8,2
2,0,8,0.8,3
3,0,9,0.8,4
4,0,10,0.8,5
...,...,...,...,...
2260,452,1,0.7,1
2261,452,5,0.7,2
2262,452,6,0.7,3
2263,452,7,0.7,4


(3-1) 각 아이템의 유사도 합 구하기

In [35]:
query = '''
SELECT item_1, sum(sim) AS SUM
FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking
FROM item_similarity) R
WHERE ranking <= {K}
GROUP BY item_1
'''.format(K = 5)
get_output(query)

,item_1,SUM
0,0,4.0
1,1,3.9
2,2,4.0
3,3,4.2
4,4,4.1
...,...,...
448,448,3.8
449,449,3.6
450,450,4.0
451,451,3.5


(3-2) 각 아이템마다 유사도를 유사도의 합으로 나누어 준다. (소수점 넷째자리까지)

In [36]:
query = '''
SELECT item_1, item_2, sim, ranking, SUM, ROUND(sim/SUM,4) AS norm_sim
FROM (SELECT item_1, item_2, sim, ranking
            FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking FROM item_similarity) R
            WHERE ranking <= {K}) A
      LEFT OUTER JOIN
      (SELECT item_1, sum(sim) AS SUM
            FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking FROM item_similarity) R
            WHERE ranking <= {K}
            GROUP BY item_1) B
      USING (item_1)
'''.format(K = 5)
get_output(query)

,item_1,item_2,sim,ranking,SUM,norm_sim
0,0,3,0.8,1,4.0,0.2000
1,0,7,0.8,2,4.0,0.2000
2,0,8,0.8,3,4.0,0.2000
3,0,9,0.8,4,4.0,0.2000
4,0,10,0.8,5,4.0,0.2000
...,...,...,...,...,...,...
2260,452,1,0.7,1,3.5,0.2000
2261,452,5,0.7,2,3.5,0.2000
2262,452,6,0.7,3,3.5,0.2000
2263,452,7,0.7,4,3.5,0.2000


(4) 아이템-사용도 평점 행렬에서 아이템의 평균으로 빈 공간 채우기

In [37]:
# 아이템 - 상용자 행렬에서 NULL 값을 평균으로 채우기
query = '''
(SELECT user, item, rating FROM ratings WHERE user = {user_id} AND rating IS NOT NULL)
UNION
(SELECT user, item, rating FROM (SELECT user, item FROM ratings WHERE user = {user_id} AND rating IS NULL) R1
          JOIN (SELECT item, AVG(rating) AS rating FROM ratings GROUP BY item) R2 USING(item))
'''.format(user_id = user)

get_output(query)



,user,item,rating
0,7,200,3.000000
1,7,259,3.000000
2,7,148,3.000000
3,7,261,3.000000
4,7,435,3.000000
...,...,...,...
448,7,448,3.215686
449,7,449,3.656250
450,7,450,2.571429
451,7,451,2.467742


(5) 행렬곱

In [38]:
# terminal 입력 부분
user = 10     # user = int(input('User Id: '))
rec_cnt = 10  # rec_cnt = int(input('Recommend Count: '))
rec_num = 3   # rec_num = float(input('Recommendation Threshold: '))

query = '''
SELECT *
FROM (SELECT item_1, item_2, sim, ranking, SUM, ROUND(sim/SUM,4) AS norm_sim
      FROM (SELECT item_1, item_2, sim, ranking
              FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking FROM item_similarity) R
              WHERE ranking <= {K}) A
           LEFT OUTER JOIN
           (SELECT item_1, sum(sim) AS SUM
              FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking FROM item_similarity) R
              WHERE ranking <= {K}
              GROUP BY item_1) B
           USING(item_1)) Item_Sim
      JOIN
      ((SELECT user, item, rating FROM ratings WHERE user = {user_id} AND rating IS NOT NULL)
        UNION
       (SELECT user, item, rating FROM (SELECT user, item FROM ratings WHERE user = {user_id} AND rating IS NULL) R1 JOIN
                      (SELECT item, AVG(rating) AS rating FROM ratings GROUP BY item) R2 USING(item))) User_Score
      ON(Item_Sim.item_2 = User_Score.item)
ORDER BY user, item_1
'''.format(K=5, user_id = user)
get_output(query)

,item_1,item_2,sim,ranking,SUM,norm_sim,user,item,rating
0,0,3,0.8,1,4.0,0.2000,10,3,3.500000
1,0,9,0.8,4,4.0,0.2000,10,9,3.000000
2,0,7,0.8,2,4.0,0.2000,10,7,3.929012
3,0,8,0.8,3,4.0,0.2000,10,8,3.928030
4,0,10,0.8,5,4.0,0.2000,10,10,3.726415
...,...,...,...,...,...,...,...,...,...
2260,452,1,0.7,1,3.5,0.2000,10,1,3.990566
2261,452,5,0.7,2,3.5,0.2000,10,5,3.718750
2262,452,6,0.7,3,3.5,0.2000,10,6,3.543860
2263,452,7,0.7,4,3.5,0.2000,10,7,3.929012


(6) 사용자 별 추천

In [39]:
# 3번 답
# terminal 입력 부분
user = 10     # user = int(input('User Id: '))
rec_cnt = 10  # rec_cnt = int(input('Recommend Count: '))
rec_num = 3   # rec_num = float(input('Recommendation Threshold: '))

query = '''
SELECT user, item_1 AS item, ROUND(SUM(norm_sim * rating),4) AS prediction
FROM (SELECT item_1, item_2, sim, ranking, SUM, ROUND(sim/SUM,4) AS norm_sim
      FROM (SELECT item_1, item_2, sim, ranking
              FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking FROM item_similarity) R
              WHERE ranking <= {K}) A
           LEFT OUTER JOIN
           (SELECT item_1, sum(sim) AS SUM
              FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC, item_2) AS ranking FROM item_similarity) R
              WHERE ranking <= {K}
              GROUP BY item_1) B
           USING(item_1)) Item_Sim
      JOIN
      ((SELECT user, item, rating FROM ratings WHERE user = {user_id} AND rating IS NOT NULL)
        UNION
       (SELECT user, item, rating FROM (SELECT user, item FROM ratings WHERE user = {user_id} AND rating IS NULL) R1 JOIN
                      (SELECT item, AVG(rating) AS rating FROM ratings GROUP BY item) R2 USING(item))) User_Score
      ON(Item_Sim.item_2 = User_Score.item)
WHERE item_1 NOT IN (SELECT item FROM ratings WHERE user = {user_id} AND rating IS NOT NULL)
GROUP BY user, item_1
HAVING prediction > {threshold}
ORDER BY prediction DESC, item
LIMIT {rec_cnt}
'''.format(K = 5, user_id = user, rec_cnt = rec_cnt,  threshold = rec_num)

get_output(query)

,user,item,prediction
0,10,325,4.5556
1,10,81,4.5000
2,10,99,4.4759
3,10,392,4.4295
4,10,224,4.4274
5,10,157,4.4165
6,10,283,4.4057
7,10,292,4.4042
8,10,192,4.4000
9,10,210,4.4000


---

## <B> 4. UBCF

In [40]:
# 4번 답
# terminal 입력 부분
user = 7     # user = int(input('User Id: '))
rec_cnt = 10  # rec_cnt = int(input('Recommend Count: '))
rec_num = 2   # rec_num = float(input('Recommendation Threshold: '))

query = '''
SELECT user_1 AS user, item, ROUND(SUM(norm_sim * rating),4) AS prediction
FROM (SELECT user_1, user_2, sim, ranking, SUM, ROUND(sim/SUM,4) AS norm_sim
      FROM (SELECT user_1, user_2, sim, ranking
              FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY user_1 ORDER BY sim DESC, user_2) AS ranking FROM user_similarity) R
              WHERE ranking <= {K}) A
           LEFT OUTER JOIN
           (SELECT user_1, sum(sim) AS SUM
              FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY user_1 ORDER BY sim DESC, user_2) AS ranking FROM user_similarity) R
              WHERE ranking <= {K}
              GROUP BY user_1) B
           USING(user_1)
      WHERE user_1 = {user_id}) User_Sim
      JOIN
      ((SELECT user, item, rating FROM ratings WHERE rating IS NOT NULL)
        UNION
       (SELECT user, item, rating FROM (SELECT user, item FROM ratings WHERE rating IS NULL) R1
          JOIN (SELECT user, AVG(rating) AS rating FROM ratings GROUP BY user) R2 USING(user))) User_Score
      ON(User_Sim.user_2 = User_Score.user)
WHERE item NOT IN (SELECT item FROM ratings WHERE user = {user_id} AND rating IS NOT NULL)
GROUP BY user, item
HAVING prediction > {threshold}
ORDER BY prediction DESC, item
LIMIT {rec_cnt}
'''.format(K = 5, user_id = user, rec_cnt = rec_cnt,  threshold = rec_num)

get_output(query)

,user,item,prediction
0,7,284,4.1443
1,7,166,4.0604
2,7,170,4.0604
3,7,173,4.0604
4,7,179,4.0604
5,7,224,4.0604
6,7,232,4.0604
7,7,234,4.0604
8,7,237,4.0604
9,7,240,4.0604


끝